In [309]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes
import base64
import random
import math
import doctest

In [318]:
def key_generator_AES(kp):
    """
    Générer une clef de 16 bytes avec kp bits significatifs
    >>> len(key_generator_AES(10))
    16
    """
    #get_random_bytes(kp)+bytes((16 - kp))
    b = get_random_bytes(math.ceil(kp/8))
    b = bytes_to_bin(b)
    if kp != len(b)-2:
        b = b[:kp+2]
    b = b.ljust(16*8 +2,'0')
    
    b = bitstring_to_bytes(b) #reconverti en bytes
    return b 

def bytes_to_bin(by):
    """
    Convertir des bytes en bits 
    quand l'entier representant le byte a moins de 8 bits,
    on le bourre de 0 à gauche
    """
    l = list(by)
    res = '0b'
    for i in l:
        res += bin(i)[2:].zfill(8)
    return res

def bitstring_to_bytes(s):
    """
    Convertir des bits en bytes
    """
    v = int(s, 2)
    b = bytearray()
    while v:
        b.append(v & 0xff)
        v >>= 8
    return bytes(b[::-1])

In [342]:
def double_AES(msg, key1, key2,  mode = 'MODE_ECB'):
    """
    Faire le double chiffrement AES
    en passant les 2 clefs key1 et key2 en parametres
    (on a pas defini les autres modes)
    """
    enc = simple_enc_AES(msg, key1)
    enc = simple_enc_AES(enc, key2)
    return enc

    
def simple_enc_AES(msg, key, mode = 'MODE_ECB'):
    """
    Chiffrer AES
    Retourne le message chiffré par l'AES en bytes
    """
    if not (isinstance(msg,bytes)):
        msg = msg.encode()
        
    cipher = AES.new(key, AES.MODE_ECB)
    enc = cipher.encrypt(pad(msg, len(key)))
    return enc

def simple_dec_AES(msg, key, mode = 'MODE_ECB'):
    """
    Dechiffrer AES
    Retourne le message non unpader et en bytes
    
    >>> msg = 'test'
    >>> key = key_generator_AES(10)
    >>> encrypt = simple_enc_AES(msg, key)
    >>> unpad(simple_dec_AES(encrypt,key),16).decode()
    'test'
    """
    cipher = AES.new(key, AES.MODE_ECB)
    plaintext = cipher.decrypt(msg)
    
    # error de padding si clef incorrevt
    #return unpad(plaintext, len(key))  
    return plaintext

In [343]:
kp = 3*8
key1 = key_generator_AES(kp)
key2 = key_generator_AES(kp)
M1 = "Voici le message 1"
C1 = double_AES(M1,key1,key2)
M2 = "Voici le message 2"
C2 = double_AES(M2,key1,key2)

In [344]:
def bin_right_padding(c,kp):
    """
    Prend les kp premiers bit de c (qui est en bytes),
    ajoute des 0 à droite pour avoir 128 bits
    et retourne le byte correspondant 
    """
    res = bytes_to_bin(c)
    res = res[:kp+2]
    res = res.ljust(16*8 +2,'0')
    res = bitstring_to_bytes(res) 
    return res

In [345]:
def trail(f, msg, kp, l):
    """
    Retourne un triplet (x0, xd, d) 
    f : fonction chiffrement OU dechiffrement
    msg : message clair OU chiffré deux fois  
    kp :nb de bit significatif de la clef
    l : nb de bit à 0 (pour la condition d'arrêt)
    """
    x0 = key_generator_AES(kp)           # clef aléatoire initiale
    tmp = x0

    d = 0                                # compter le nb de pas
    max_it = (20/ (1/(2**l))) // 3       # diviser par 3 car sinon ca prends tres longtemps
    mask_l = 2**l - 1               
    
    while True:
        
        if len(tmp) < 16:                # des fois les 8 premiers bits sont nuls alors la clef devient trop courte 
            tmp = tmp.rjust(16,b"\x00")  # après la conversion de bits en byte
        c_temp = f(msg, tmp)
        
        if d == max_it:
            #print("Risque de cycle ")
            return None
        
        tmp_binary = bytes_to_bin(tmp)[:kp+2]
        if int(tmp_binary,2) & mask_l == 0:  # condition d'arret
            xd = tmp
            break
            
        d += 1
        
        tmp = bin_right_padding(c_temp,kp)         # générer la nouvelle clef       
    return (x0, xd, d)



In [346]:
kp = 2*8
l = 5
print(trail(simple_enc_AES, M1, kp, l ))

(b'D\xd7\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', b'\x97\xa0\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 45)


In [363]:
def F(b):
    """
    Choisir une fonction
    b : 0 OU 1
    0 correspond à simple_enc_AES
    1 correspond à simple_dec_AES
    """
    if b == 0:
        return simple_enc_AES
    if b == 1:
        return simple_dec_AES

In [364]:
def new_step(f, M, kp, x):
    """
    passer d'un xi au suivant
    """
    if len(x) < 16:                     # des fois les 8 premiers bits sont nuls
        x = x.rjust(16,b"\x00")
        
    c = f(M, x)
    
    tmp = bin_right_padding(c,kp)
    return tmp

In [365]:
def remonter (F, A, B, M, C, kp, b):
    """
    Returne ( (x, f1) , (y, f2) ) tq x != y et f1(x) == f2(y)
    F : choix entre chiffrement et déchiffrement 
    A, B : triplet (x0, xd, d)
    M, C : clair et chiffré double
    kp : nb de bits significatif de la clef
    b : 0 ou 1
    """       
    couple = [M,C]
    
    if A[2] >= B[2]: 

        x = A[0]
        for _ in range(A[2]-B[2]):
            x =  new_step(F(b), couple[b], kp, x)
        y = B[0]

        if x == y : 
            #print('pb : x==y et fhash(x)==fhash(y)')
            return None

        while True:
            if x == y :
                break
            tmp1 = x            
            tmp2 = y             # anciennes valeurs
            x =  new_step(F(b), couple[b], kp, tmp1)
            y = new_step(F(1-b), couple[1-b], kp, tmp2)
            
        return (   (tmp1, F(b)) , (tmp2, F(1-b))    )
        
        
    else:          # A[2] < B[2] mais on fait la meme chose
        y = B[0]
        for _ in range(B[2]-A[2]):
            y =  new_step(F(1-b), couple[1-b], kp, y)
        x = A[0]

        if x == y : 
            #print('pb : x==y et fhash(x)==fhash(y)')
            return None

        while True:
            if x == y :
                break

            tmp1 = x            
            tmp2 = y             # anciennes valeurs
            x = new_step(F(b), couple[b], kp, tmp1)
            y = new_step(F(1-b), couple[1-b], kp, tmp2)

        return (   (tmp1, F(b)) , (tmp2, F(1-b))    )

In [366]:
def collision_detection(F, M, C, kp, l, dico):
    """
    Detecte une seule collision
    Retourne le couple ( (x, f1) , (y, f2) ) tq x != y et f1(x) == f2(y)
    F : choix entre chiffrement et déchiffrement 
    M, C : clair et chiffré double
    kp : nb de bits significatif de la clef
    l : nb de bits pour la condition d'arrêt
    dico : contenant les collisions trouvées
    """
    couple = [M,C]
    seuil = 10000                  # valeur estimé pour le nb max d'éléments dans le dico
    
    while True: 
        b = random.randint(0,1) 
        res = trail(F(b),couple[b], kp, l)
        
        if res == None:
            continue
            
        x0, xd, d = res

        if (xd,1-b) in dico:        # collision trouvée
            A = (x0,xd,d)                                   # b
            B = (dico[(xd,1-b)][0], xd, dico[(xd,1-b)][1])  # 1-b
            return remonter(F, A , B, M, C, kp, b )
        
        # si la taille du dico depasse la limite fixée, choix aléatoire de la victime
        if len(dico) >= seuil:      
            delete = random.choice(list(dico.keys()))
            dico.pop(delete)
        dico[(xd,b)] = (x0, d)

In [367]:
dico={}
collision_detection(F, M1, C1, kp, l,dico)

((b'\xa9\xd3\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00',
  <function __main__.simple_enc_AES(msg, key, mode='MODE_ECB')>),
 (b'\x0el\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00',
  <function __main__.simple_dec_AES(msg, key, mode='MODE_ECB')>))

In [261]:
def golden_collision(F, M1, C1, M2, C2 ,kp, l):
    """
    Trouve la golden collision en vérifiant à chaque fois
    les clefs obtenus avec (M2, C2)
    F : choix entre chiffrement et déchiffrement 
    M1, C1 : clair et chiffré double pour trouver les collisions
    M2, C2 : clair et chiffré double pour la vérificaiton
    kp : nb de bits significatif de la clef
    l : nb de bits pour la condition d'arrêt
    """
    dico = {}
    liste = []
    while True:
    
        colli = collision_detection(F, M1, C1, kp, l,dico)
        if colli == None:
            continue
        if colli in liste or (colli[1],colli[0]) in liste: # si collision deja trouvé 
            print('idem colli')
            continue 
        print('new colli')
        liste.append(colli)
        if (len(liste)==2**kp):
            print('GROOS PB !!!!!')
            break
        try: 
            if colli[0][1].__name__ == "simple_enc_AES":
                tmp1 = colli[0][1](M2, colli[0][0])
                tmp2 = unpad(colli[1][1](C2, colli[1][0]), 16)
            else:
                tmp1 = unpad(colli[0][1](C2, colli[0][0]),16)
                tmp2 = colli[1][1](M2, colli[1][0])
            if( tmp1 == tmp2):
                print( "GOLDEN COLLISION ! ")
                return (colli)
        except ValueError:
            pass


In [369]:
kp = 16
key1 = key_generator_AES(kp)
key2 = key_generator_AES(kp)
M1 = "Voici le message 1"
C1 = double_AES(M1,key1,key2)
M2 = "Voici le message 2"
C2 = double_AES(M2,key1,key2)

print(key1)
print(key2)

b'\xa9[\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
b'a\xad\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'


In [ ]:
l=4

golden_collision(F, M1, C1,M2,C2 ,kp, l )

new colli
new colli
new colli
new colli
new colli
new colli
new colli
new colli
new colli
new colli
new colli
idem colli
new colli
idem colli
new colli
idem colli
new colli
new colli
new colli
new colli
new colli
new colli
new colli
new colli
new colli
new colli
idem colli
idem colli
new colli
new colli
new colli
idem colli
idem colli
new colli
new colli
new colli
new colli
idem colli
idem colli
new colli
idem colli
idem colli
idem colli
new colli
idem colli
new colli
idem colli
idem colli
new colli
idem colli
new colli
idem colli
new colli
idem colli
new colli
new colli
new colli
idem colli
idem colli
new colli
new colli
idem colli
new colli
idem colli
idem colli
new colli
new colli
new colli
idem colli
idem colli
idem colli
idem colli
idem colli
new colli
idem colli
new colli
idem colli
new colli
idem colli
new colli
new colli
new colli
idem colli
new colli
new colli
new colli
new colli
new colli
new colli
new colli
idem colli
new colli
new colli
idem colli
new colli
new colli
new co

idem colli
idem colli
idem colli
new colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
new colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
new colli
new colli
new colli
idem colli
new colli
new colli
new colli
new colli
idem colli
new colli
idem colli
idem colli
new colli
new colli
idem colli
idem colli
new colli
new colli
new colli
new colli
idem colli
idem colli
idem colli
new colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
new colli
idem colli
new colli
new colli
idem colli
new colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
new colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
new colli
new colli
new colli
idem colli
idem colli
idem colli
idem colli
new colli
idem colli
new colli
idem colli
idem

new colli
new colli
idem colli
idem colli
idem colli
new colli
idem colli
idem colli
new colli
new colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
new colli
idem colli
idem colli
new colli
new colli
idem colli
idem colli
idem colli
idem colli
idem colli
new colli
idem colli
idem colli
new colli
idem colli
new colli
idem colli
idem colli
idem colli
idem colli
new colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
new colli
idem colli
idem colli
new colli
idem colli
new colli
idem colli
idem colli
idem colli
new colli
idem colli
new colli
idem colli
idem colli
new colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
new colli
new colli
new colli
new colli
new colli
idem colli
idem colli
idem colli
idem colli
new colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
i

idem colli
idem colli
idem colli
idem colli
idem colli
new colli
idem colli
idem colli
idem colli
idem colli
idem colli
new colli
new colli
idem colli
idem colli
idem colli
idem colli
new colli
idem colli
idem colli
idem colli
idem colli
new colli
idem colli
new colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
new colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
new colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
new colli
idem colli
idem colli
new colli
idem colli
idem colli
idem colli
idem colli
new colli
idem colli
idem colli
idem colli
idem colli
new colli
idem colli
idem colli
new colli
idem colli
idem colli
new colli
idem colli
idem colli
new colli
new colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
new colli
idem colli
idem colli
idem colli
idem 

idem colli
new colli
idem colli
new colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
new colli
new colli
idem colli
idem colli
idem colli
new colli
idem colli
idem colli
idem colli
new colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
new colli
new colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
new colli
idem colli
idem colli
new colli
idem colli
new colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
new colli
idem colli
new colli
new colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
new colli
idem colli
idem colli
idem colli
idem colli
idem colli
new colli
idem colli
idem colli
idem colli
idem colli
idem colli
new colli
new colli
idem colli
idem colli
new colli
idem colli
idem colli
new colli
idem colli
idem colli
idem colli
idem colli
idem col

...

idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
new colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
new colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
new colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
id

idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
new colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
new colli
idem colli
new colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
id

idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli

idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
new colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
new colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
new colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
id

idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
new colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
idem colli
